# LTCM 

*Case: Long-Term Capital Management, L.P. (A) [9-200-007].*

***

# 2. Fund Performance and Attribution

### Data

* `ltcm exhibits data.xlsx`, `Exhibit 2`: Gross and net (total) returns of LTCM
* `spy_data.xlsx`: SPY returns and risk-free rate (scaled tbill index)

In [6]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

FILE_LTCM = '../data/ltcm_exhibits_data.xlsx'
FILE_SPY = '../data/spy_data.xlsx'
MONTHS_PER_YEAR = 12


def load_ltc_data(path):
    df = pd.read_excel(path, sheet_name='Exhibit 2', header=2)
    df = df.rename(columns={df.columns[0]: 'date'})
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df = df.dropna(subset=['date']).set_index('date')
    df.index = df.index.to_period('M').to_timestamp('M')
    df = df[['Fund Capital ($billions)', 'Gross Monthly Performancea',
             'Net Monthly Performanceb', 'Index of Net Performance']].astype(float)
    df.columns = ['capital', 'gross', 'net', 'net_index']
    return df


def load_spy_data(path):
    df = pd.read_excel(path, sheet_name='total returns', index_col=0, parse_dates=True)
    df.index = df.index.to_period('M').to_timestamp('M')
    return df[['SPY', '^IRX']]


def summarize(series):
    return pd.Series({
        'mean(ann)': series.mean() * MONTHS_PER_YEAR,
        'vol(ann)': series.std(ddof=0) * np.sqrt(MONTHS_PER_YEAR),
        'sharpe': np.nan if series.std(ddof=0) == 0 else (series.mean() * np.sqrt(MONTHS_PER_YEAR)) / series.std(ddof=0),
        'skew': series.skew(),
        'kurtosis': series.kurtosis(),
        'q5': series.quantile(0.05)
    })

ltcm = load_ltc_data(FILE_LTCM)
spy = load_spy_data(FILE_SPY)
combined = ltcm.join(spy, how='inner')
gross_excess = combined['gross'] - combined['^IRX']
net_excess = combined['net'] - combined['^IRX']
spy_excess = combined['SPY'] - combined['^IRX']
print(f"Working sample spans {combined.index.min():%Y-%m} to {combined.index.max():%Y-%m} with {len(combined)} observations.")


Working sample spans 1994-03 to 1998-07 with 53 observations.


### 1. Summary stats.

For both the gross and net series of LTCM excess returns, report the annualized 
* mean
* volatility
* Sharpe ratios

Also report the
* skewness
* kurtosis
* 5th quantile


In [7]:
stats_table = pd.DataFrame({
    'LTCM gross excess': summarize(gross_excess),
    'LTCM net excess': summarize(net_excess),
    'SPY excess': summarize(spy_excess)
})
display(stats_table.loc[['mean(ann)', 'vol(ann)', 'sharpe']].T)
display(stats_table.loc[['skew', 'kurtosis', 'q5']].T)


,mean(ann),vol(ann),sharpe
LTCM gross excess,0.243600,0.134946,1.805161
LTCM net excess,0.156883,0.110713,1.417022
SPY excess,0.154775,0.112991,1.369790


,skew,kurtosis,q5
LTCM gross excess,-0.288328,1.586681,-0.030305
LTCM net excess,-0.810870,2.927724,-0.026300
SPY excess,-0.406867,-0.388002,-0.049667



### 2. Compare to SPY

Comment on how these stats compare to SPY and other assets we have seen. 

How much do they differ between gross and net?


Gross LTCM excess returns delivered roughly 24% per year with 13.5% volatility, yielding a Sharpe near 1.81—almost 40 bps of extra mean and modestly lower risk than SPY’s 15.5%/11.3% profile (Sharpe ≈1.37). Net of fees, the strategy still compounded around 15.7% with 11.1% vol for a 1.42 Sharpe, so the fee drag costs roughly 8–9 bps per month and trims volatility by ~2.4 points. Tail moments stay unfavorable for LTCM (skew −0.29 gross, −0.81 net versus SPY’s −0.41) and the gross series exhibits higher kurtosis (1.59 vs −0.39 for SPY), but its 5% VaR (−3.0%) is slightly better than SPY’s −5.0%. Overall, LTCM outperformed SPY on a mean/vol basis, yet investors faced more negative skew and fat tails, and the difference between gross and net aligns with the case’s performance fee structure.


### 3. LFD

Estimate a linear factor decomposition of **net** LTCM excess returns on `SPY` excess returns.

Report
* annualized alpha
* beta
* r-squared

Does LTCM deliver performance beyond `SPY`?

$$\newcommand{\betalinear}{\beta_{\text{linear}}}
\newcommand{\betaquad}{\beta_{\text{quad}}}
$$

In [8]:
lfd_X = sm.add_constant(spy_excess.rename('SPY'))
lfd_res = sm.OLS(net_excess, lfd_X, missing='drop').fit()
lfd_summary = pd.Series({
    'alpha (ann)': lfd_res.params['const'] * MONTHS_PER_YEAR,
    'beta(SPY)': lfd_res.params['SPY'],
    'r2': lfd_res.rsquared
})
display(lfd_summary.to_frame('Net excess vs SPY').T)


,alpha (ann),beta(SPY),r2
Net excess vs SPY,0.135076,0.140892,0.020676


The LFD shows LTCM still posts roughly 13.5% annualized alpha with only ~0.14 beta to SPY and an r² of 2%. The slope is small enough that broad equity swings explain little of the action—spread convergence remains the dominant driver even after 1998’s turbulence.


### 4. Nonlinear Exposure

Let's check for non-linear market exposure. Run the following regression on LTCM's **net** excess returns:

$$
\tilde{r}_t^{\text{ltcm}} = \alpha + \betalinear \tilde{r}_t^m + \betaquad \left(\tilde{r}_t^m\right)^2 + \epsilon_t
$$

Report 
* annualized alpha
* the linear and quadratic betas
* r-squared

In [9]:
quad_X = pd.DataFrame({
    'SPY': spy_excess,
    'SPY^2': spy_excess ** 2
})
quad_res = sm.OLS(net_excess, sm.add_constant(quad_X), missing='drop').fit()
quad_summary = pd.Series({
    'alpha (ann)': quad_res.params['const'] * MONTHS_PER_YEAR,
    'beta(SPY)': quad_res.params['SPY'],
    'beta(SPY^2)': quad_res.params['SPY^2'],
    'r2': quad_res.rsquared
})
display(quad_summary.to_frame('Quadratic LFD').T)


,alpha (ann),beta(SPY),beta(SPY^2),r2
Quadratic LFD,0.162629,0.168741,-2.158255,0.027801


### 5. 

* Does the quadratic market factor do much to increase the overall LTCM variation explained by the market?
* From the regression evidence, does LTCM's market exposure behave as if it is long market options or short market options?
* Should we describe LTCM as being positively or negatively exposed to market volatility?

Adding the quadratic market term nudges alpha up to ~16% and lifts the linear beta to 0.17, but the −2.16 coefficient on $r_m^2$ signals a short-convexity profile: performance fades when markets move far from zero. The r² only improves to about 2.8%, so most variance is still outside the market factor, yet the negative curvature flags the liquidity- provider risk embedded in LTCM’s trades.


### 6. 

Let's try to pinpoint the nature of LTCM's nonlinear exposure. Does it come more from exposure to up-markets or down-markets? Run the following regression on LTCM's net excess returns:

$$
\tilde{r}_t^{\text{ltcm}}  = \alpha + \beta\tilde{r}_t^m + \beta_u \max\left(\tilde{r}_t^m-k_1,0\right) + \beta_d \max\left(k_2 - \tilde{r}_t^m\right) + \epsilon_t
$$

where $k_1= .03$ and $k_2= -.03$. 

Report 
* annualized alpha
* market beta, the **up** and **down** betas
* r-squared

In [10]:
k_up = 0.03
k_down = -0.03
rm = spy_excess
up_tail = np.maximum(rm - k_up, 0)
down_tail = np.maximum(k_down - rm, 0)
updown_X = pd.DataFrame({
    'SPY': rm,
    'up_tail': up_tail,
    'down_tail': down_tail
})
updown_res = sm.OLS(net_excess, sm.add_constant(updown_X), missing='drop').fit()
updown_summary = pd.Series({
    'alpha (ann)': updown_res.params['const'] * MONTHS_PER_YEAR,
    'beta(SPY)': updown_res.params['SPY'],
    'beta(up)': updown_res.params['up_tail'],
    'beta(down)': updown_res.params['down_tail'],
    'r2': updown_res.rsquared
})
display(updown_summary.to_frame('Piecewise LFD').T)


,alpha (ann),beta(SPY),beta(up),beta(down),r2
Piecewise LFD,0.109438,0.434499,-0.721876,1.042253,0.048607


### 7.

* Is LTCM long or short the call-like factor? And the put-like factor?
* Which factor moves LTCM more, the call-like factor, or the put-like factor?
* In the previous problem, you commented on whether LTCM is positively or negatively exposed to market volatility. Using this current regression, does this volatility exposure come more from being long the market's upside? Short the market's downside? Something else?

The piecewise regression confirms that volatility exposure comes primarily from the downside. The base beta jumps to ~0.43, but the “up” shock loads at −0.72 (short calls) while the “down” shock loads at +1.04 (long puts in payout terms, i.e., losses when markets drop). Even though r² rises to ~4.9%, most outcomes remain idiosyncratic; however, the asymmetric betas make clear LTCM sacrifices upside rallies and is highly sensitive to sharp selloffs.
